In [1]:
import pandas as pd
import numpy as np
import torch
import wandb
from datasets import Dataset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from lifelines import KaplanMeierFitter, CoxPHFitter, NelsonAalenFitter
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [2]:
train = pd.read_csv("../data/train.csv")

In [3]:
cat_cols = []
num_cols = []
RMV = ["ID", "efs", "efs_time", "target"]
FEATURES = [c for c in train.columns if not c in RMV]
print(f"There are {len(FEATURES)} FEATURES: {FEATURES}")

for c in FEATURES:
    if train[c].dtype == "object" or train[c].dtype == "category":
        cat_cols.append(c)
    else:
        num_cols.append(c)
print(f"In these features, there are {len(cat_cols)} CATEGORICAL FEATURES: {cat_cols}")

There are 57 FEATURES: ['dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'hla_match_c_high', 'hla_high_res_8', 'tbi_status', 'arrhythmia', 'hla_low_res_6', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe', 'prim_disease_hct', 'hla_high_res_6', 'cmv_status', 'hla_high_res_10', 'hla_match_dqb1_high', 'tce_imm_match', 'hla_nmdp_6', 'hla_match_c_low', 'rituximab', 'hla_match_drb1_low', 'hla_match_dqb1_low', 'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity', 'year_hct', 'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hla_match_a_high', 'hepatic_severe', 'donor_age', 'prior_tumor', 'hla_match_b_low', 'peptic_ulcer', 'age_at_hct', 'hla_match_a_low', 'gvhd_proph', 'rheum_issue', 'sex_match', 'hla_match_b_high', 'race_group', 'comorbidity_score', 'karnofsky_score', 'hepatic_mild', 'tce_div_match', 'donor_related', 'melphalan_dose', 'hla_low_res_8', 'cardiac', 'hla_match_drb1_high', 'pulm_moderate', 'hla_low_res_10']
In these features, there are 35 CATEGORI

In [4]:
def update_target_with_survival_probabilities(df, method="kaplan", time_col="efs_time", event_col="efs"):
    res = np.zeros(df.shape[0])
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    for train_idx, val_idx in skf.split(df, df["race_group"]):
        X_trn, X_val = df.iloc[train_idx], train.iloc[val_idx]
        if method == "kaplan":
            kmf = KaplanMeierFitter()
            kmf.fit(durations=X_trn[time_col], event_observed=X_trn[event_col])
            res[val_idx] = kmf.survival_function_at_times(X_val[time_col]).values
        elif method == "nelson":
            naf = NelsonAalenFitter()
            naf.fit(durations=X_trn[time_col], event_observed=X_trn[event_col])
            res[val_idx] = -naf.cumulative_hazard_at_times(X_val[time_col]).values
        else:
            data_trn = pd.get_dummies(X_trn, columns=cat_cols, drop_first=True).drop("ID", axis=1)
            data_val = pd.get_dummies(X_val, columns=cat_cols, drop_first=True).drop("ID", axis=1)
            train_data = data_trn.loc[:, data_trn.nunique() > 1]
            valid_data = data_val[train_data.columns]
            cph = CoxPHFitter(penalizer=0.01)
            cph.fit(train_data, duration_col=time_col, event_col=event_col)
            res[val_idx] = cph.predict_partial_hazard(valid_data).values
    df["target"] = res
    df.loc[df[event_col] == 0, "target"] -= 0.15
    return df

In [5]:
train = update_target_with_survival_probabilities(train, method="kaplan", time_col="efs_time", event_col="efs")
train.head()

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time,target
0,0,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,...,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,42.356,0.307494
1,1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,...,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1.0,4.672,0.847708
2,2,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,19.793,0.312222
3,3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,...,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,102.349,0.306520
4,4,High,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,Related,MEL,8.0,No,2.0,No,10.0,0.0,16.223,0.314988


In [6]:
def update(df):
    global cat_cols
    for c in cat_cols:
        df[c] = df[c].astype(str).fillna("Unknown").astype("category")
    for c in num_cols:
        if df[c].dtype == "float64":
            df[c] = df[c].fillna(0).astype("float32")
        if df[c].dtype == "int64":
            df[c] = df[c].fillna(0).astype("int32")
    j_ch = ',[]{}:"\\<'
    for ch in j_ch:
        for c in cat_cols:
            df[c] = df[c].apply(lambda x: str(x).replace(ch, ""))
    return df


train = update(train)

In [7]:
RMV = ["ID", "efs", "efs_time", "target"]
FEATURES = [c for c in train.columns if not c in RMV]


def create_text(row):
    text = []
    for col in FEATURES:
        text.append(f"{col}: {row[col]}")
    return "\n".join(text)


train["text"] = train.apply(create_text, axis=1)
train["text"]

0        dri_score: N/A - non-malignant indication\npsy...
1        dri_score: Intermediate\npsych_disturb: No\ncy...
2        dri_score: N/A - non-malignant indication\npsy...
3        dri_score: High\npsych_disturb: No\ncyto_score...
4        dri_score: High\npsych_disturb: No\ncyto_score...
                               ...                        
28795    dri_score: Intermediate - TED AML case missing...
28796    dri_score: High\npsych_disturb: No\ncyto_score...
28797    dri_score: TBD cytogenetics\npsych_disturb: na...
28798    dri_score: N/A - non-malignant indication\npsy...
28799    dri_score: N/A - pediatric\npsych_disturb: No\...
Name: text, Length: 28800, dtype: object

In [8]:
train.head()

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time,target,text
0,0,N/A - non-malignant indication,No,nan,No,0.0,0.0,No TBI,No,6.0,...,N/A Mel not given,8.0,No,2.0,No,10.0,0.0,42.356,0.307494,dri_score: N/A - non-malignant indication\npsy...
1,1,Intermediate,No,Intermediate,No,2.0,8.0,TBI +- Other >cGy,No,6.0,...,N/A Mel not given,8.0,No,2.0,Yes,10.0,1.0,4.672,0.847708,dri_score: Intermediate\npsych_disturb: No\ncy...
2,2,N/A - non-malignant indication,No,nan,No,2.0,8.0,No TBI,No,6.0,...,N/A Mel not given,8.0,No,2.0,No,10.0,0.0,19.793,0.312222,dri_score: N/A - non-malignant indication\npsy...
3,3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,...,N/A Mel not given,8.0,No,2.0,No,10.0,0.0,102.349,0.306520,dri_score: High\npsych_disturb: No\ncyto_score...
4,4,High,No,nan,No,2.0,8.0,No TBI,No,6.0,...,MEL,8.0,No,2.0,No,10.0,0.0,16.223,0.314988,dri_score: High\npsych_disturb: No\ncyto_score...


In [9]:
skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
for i, (train_index, val_index) in enumerate(skf.split(train, train["race_group"])):
    train.loc[val_index, "fold"] = int(i)
train.head()

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time,target,text,fold
0,0,N/A - non-malignant indication,No,nan,No,0.0,0.0,No TBI,No,6.0,...,8.0,No,2.0,No,10.0,0.0,42.356,0.307494,dri_score: N/A - non-malignant indication\npsy...,6.0
1,1,Intermediate,No,Intermediate,No,2.0,8.0,TBI +- Other >cGy,No,6.0,...,8.0,No,2.0,Yes,10.0,1.0,4.672,0.847708,dri_score: Intermediate\npsych_disturb: No\ncy...,5.0
2,2,N/A - non-malignant indication,No,nan,No,2.0,8.0,No TBI,No,6.0,...,8.0,No,2.0,No,10.0,0.0,19.793,0.312222,dri_score: N/A - non-malignant indication\npsy...,3.0
3,3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,...,8.0,No,2.0,No,10.0,0.0,102.349,0.306520,dri_score: High\npsych_disturb: No\ncyto_score...,7.0
4,4,High,No,nan,No,2.0,8.0,No TBI,No,6.0,...,8.0,No,2.0,No,10.0,0.0,16.223,0.314988,dri_score: High\npsych_disturb: No\ncyto_score...,9.0


In [10]:
train["fold"].value_counts()

fold
6.0    2880
5.0    2880
3.0    2880
7.0    2880
9.0    2880
2.0    2880
8.0    2880
1.0    2880
4.0    2880
0.0    2880
Name: count, dtype: int64

In [11]:
train_df = train[train["fold"] != 0.0]
train_df = train_df[["text", "target"]].sample(frac=1.0, random_state=42)
val_df = train[train["fold"] == 0.0]
val_df = val_df[["text", "target"]].sample(frac=1.0, random_state=42)
print(f"Train shape: {train_df.shape}, Val shape: {val_df.shape}")
train_df.head()

Train shape: (25920, 2), Val shape: (2880, 2)


,text,target
22473,dri_score: High\npsych_disturb: No\ncyto_score...,0.310347
6165,dri_score: Intermediate\npsych_disturb: Yes\nc...,0.307762
1504,dri_score: Low\npsych_disturb: No\ncyto_score:...,0.496483
8389,dri_score: N/A - pediatric\npsych_disturb: No\...,0.519440
25461,dri_score: Intermediate\npsych_disturb: No\ncy...,0.714656


In [ ]:
MAX_LENGTH = 2048
MODEL_NAME = "microsoft/deberta-v3-base"
LEARNING_RATE = 5e-5
BATCH_SIZE = 1
ACCUMULATION_STEPS = 16
WARMUP_RATIO = 0.1
EPOCHS = 3
WEIGHT_DECAY = 0.0001
FOLD_ID = 0
STEPS = 100
SAVE_TOTAL_LIMIT = 10
LR_SCHEDULER = "cosine"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


def tokenize(sample):
    return tokenizer(sample["text"], max_length=MAX_LENGTH, truncation=True)


ds_train = Dataset.from_pandas(train_df)
ds_val = Dataset.from_pandas(val_df)

ds_train = ds_train.map(tokenize).remove_columns(["text"])
ds_val = ds_val.map(tokenize).remove_columns(["text"])

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=1,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True
)


class DataCollator:
    def __call__(self, features):
        model_inputs = [
            {
                "input_ids": feature["input_ids"],
                "attention_mask": feature["attention_mask"],
                "labels": feature["target"]
            } for feature in features
        ]
        batch = tokenizer.pad(
            model_inputs,
            padding="max_length",
            max_length=MAX_LENGTH,
            return_tensors="pt",
            pad_to_multiple_of=8
        )
        return batch


def compute_metrics(p):
    preds, labels = p
    preds = preds.astype(np.float32)
    labels = labels.astype(np.float32)
    mse = mean_squared_error(labels, preds)
    return {"mse": mse}


wandb.login(key="96a47264bf4a345cddba37487838a3c098362dab")
run = wandb.init(project=f"{MODEL_NAME.split('/')[-1]}", job_type="training", anonymous="allow")

training_args = TrainingArguments(
    output_dir=f"output_{MODEL_NAME.split('/')[-1]}_{wandb.run.name}/Fold{FOLD_ID}",
    bf16=True if torch.cuda.is_bf16_supported() else False,
    fp16=False if torch.cuda.is_bf16_supported() else True,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE * 2,
    gradient_accumulation_steps=ACCUMULATION_STEPS,
    warmup_ratio=WARMUP_RATIO,
    optim="paged_adamw_8bit",
    num_train_epochs=EPOCHS,
    weight_decay=WEIGHT_DECAY,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=STEPS,
    save_total_limit=SAVE_TOTAL_LIMIT,
    save_strategy="steps",
    save_steps=STEPS,
    logging_steps=STEPS,
    load_best_model_at_end=True,
    metric_for_best_model="mse",
    greater_is_better=False,
    save_only_model=True,
    lr_scheduler_type=LR_SCHEDULER,
    gradient_checkpointing=False,
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
    tokenizer=tokenizer,
    data_collator=DataCollator(),
    compute_metrics=compute_metrics
)

trainer.train()
wandb.finish()